# Installing Dependencies

In [ ]:
!pip install -r requirements.txt

# Loading Previous Data

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from problems.data.data_preprocessing import get_kuka_data
from dkibo import DKIBO, UtilityFunction

DATA_DIR = r'.\problems\data'
previous_data = get_kuka_data(DATA_DIR)
bounds = {}
for key in previous_data.columns[1:]:
    bounds[key] = (0, 5)

optimizer = DKIBO(f=None, pbounds=bounds, ml_regressor=LinearRegression(), use_noise=True)

target = previous_data['target'].to_numpy()
params = previous_data[previous_data.columns[1:]]
for i in range(len(target)):
    try:
        optimizer.register(params.iloc[i].to_numpy(), target[i])
    except KeyError as e:
        print(e, ', skipping this point')
        
util = UtilityFunction(kind='ucb',
                       kappa=2.576,
                       xi=0.0,
                       kappa_decay=1,
                       kappa_decay_delay=0,
                       ml_regressor=optimizer.ml_regressor,
                       x_init=optimizer.space.params[:5],
                       max_iter=100)
x_probe_previous = None
early_stop = False

# Training Model and Generate Next Point

In [ ]:
x_probe = optimizer.suggest(util)
x_probe_numpy = np.array(list(x_probe.values()))

print('Next point: ')
for key in x_probe.keys():
    print(key, ': ', x_probe[key])
    
# Early dropping
if x_probe_previous is not None:
    dis1 = np.linalg.norm(x_probe_numpy - np.mean(optimizer.space.params.mean(axis=0)))
    dis2 = np.linalg.norm(x_probe_numpy - x_probe_previous)
    early_stop = dis1 * optimizer.early_stop_threshold > dis2
    
x_probe_previous = x_probe_numpy
util.update_params(early_stop=early_stop)

# Registering Next Point

In [ ]:
# Register new point result
import ipywidgets as widgets
from IPython.display import display

def get_input(sender):
    return sender.value

text_boxes = []

x_register = {}
for key in x_probe.keys():
    text = widgets.FloatText(
        description = key,
    )
    display(text)
    text_boxes.append(text)
    
her = widgets.FloatText(
        description = 'HER/target',
    )
display(her)

In [ ]:
x_reg = []
for box in text_boxes:
    x_reg.append(box.value)
x_reg = x_reg